<a href="https://colab.research.google.com/github/wangzhenyagit/myColab/blob/main/ragas_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install openai

In [5]:

from openai import OpenAI
import os
import json

os.environ["OPENAI_API_KEY"] = "your-api-key"

client = OpenAI()

In [ ]:
# Upload a file with an "assistants" purpose
file = client.files.create(
  file=open("AutoGen- Enabling Next-Gen LLM Applications via Multi-Agent Conversation.pdf", "rb"),
  purpose='assistants'
)

In [ ]:
# Add the file to the assistant
assistant = client.beta.assistants.create(
  name="Paper Analyst",
  instructions="You are a paper analysis chatbot. Use your knowledge base to best respond to user queries.",
  model="gpt-4-1106-preview",
  tools=[{"type": "retrieval"}],
  file_ids=[file.id]
)

In [ ]:
thread = client.beta.threads.create()

In [ ]:
print(thread)

Thread(id='thread_7XZM8Qdpb8vbKowiEW5BURw3', created_at=1699706096, metadata={}, object='thread')


In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Who create AutoGen?"
)

In [ ]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [ ]:
print(run)

Run(id='run_HV5DWHVY7KJtQaDhWVV4328I', assistant_id='asst_mIK5Gb71yb1Iu3PmePtiXOby', cancelled_at=None, completed_at=None, created_at=1699707885, expires_at=1699708485, failed_at=None, file_ids=['file-xpVHErHEAvwvDcy03y36vder'], instructions='You are a paper analysis chatbot. Use your knowledge base to best respond to user queries.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_7XZM8Qdpb8vbKowiEW5BURw3', tools=[ToolAssistantToolsRetrieval(type='retrieval')])


In [ ]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

In [ ]:
print(run)

Run(id='run_HV5DWHVY7KJtQaDhWVV4328I', assistant_id='asst_mIK5Gb71yb1Iu3PmePtiXOby', cancelled_at=None, completed_at=1699707892, created_at=1699707885, expires_at=None, failed_at=None, file_ids=['file-xpVHErHEAvwvDcy03y36vder'], instructions='You are a paper analysis chatbot. Use your knowledge base to best respond to user queries.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1699707885, status='completed', thread_id='thread_7XZM8Qdpb8vbKowiEW5BURw3', tools=[ToolAssistantToolsRetrieval(type='retrieval')])


In [ ]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [21]:
print(messages.data[0].content[0].text.value)

AutoGen was created by a team of researchers, including Qingyun Wu, Gagan Bansal, Jieyu Zhang, Yiran Wu, Beibin Li, Erkang Zhu, Li Jiang, Xiaoyun Zhang, Shaokun Zhang, Jiale Liu, Ahmed Awadallah, Ryen W. White, Doug Burger, and Chi Wang【12†source】.
